In [61]:
#importer vos librairies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import datetime as DT
import time

In [3]:
class DataHandler:
    def __init__(self) :
        self.dataprice = None
        self.datalisting = None
        self.merge = None
    def get_data(self):
        print("_ _ Fetch Data From bucket _ _")
        self.dataprice=pd.read_csv("price_availability.csv",sep=';')  # What is csv : comma separateur virgule
        self.datalisting=pd.read_csv("listings_final.csv",sep=';')
        return "_ _  data loaded _ _\nFiles : \n - listing {} \n - prices {} ".format(self.datalisting.shape,self.dataprice.shape)
    def group_data(self):
        data = self.dataprice.groupby('listing_id')['local_price'].mean()
        self.merge = pd.merge(data, self.datalisting, on='listing_id')
        print(" - - - data merged - - - ")
    def get_process_data(self):
        self.get_data()
        self.group_data()
        print(self.merge)
        print("-- End of DataHandling --") 

In [4]:
d = DataHandler()

In [5]:
d.get_process_data()
%time

_ _ Fetch Data From bucket _ _
 - - - data merged - - - 
     listing_id  local_price  Unnamed: 0  \
0         56093   170.000000          12   
1         57207    49.952756          13   
2        114543   107.374026          19   
3        149534   169.000000           9   
4        164255    75.876209          28   
..          ...          ...         ...   
994    28684174   725.175781         662   
995    28709644   475.000000         745   
996    28751412   117.000000          88   
997    28774896   156.397468         159   
998    28792796    49.184211         998   

                                                  name          type   city  \
0                          Beau duplex dans le Marais    entire_home  Paris   
1                             Belle Chambre pour court  private_room  Paris   
2                    Charming 1bdr 55m² - Eiffel Tower   entire_home  Paris   
3                   GREAT WARM FULL APT LE HAUT MARAIS   entire_home  Paris   
4                  

In [6]:
d.merge.dtypes

listing_id                    int64
local_price                 float64
Unnamed: 0                    int64
name                         object
type                         object
city                         object
neighborhood                 object
latitude                    float64
longitude                   float64
person_capacity               int64
beds                          int64
bedrooms                      int64
bathrooms                   float64
is_rebookable                  bool
is_new_listing                 bool
is_fully_refundable            bool
is_host_highly_rated           bool
is_business_travel_ready       bool
pricing_weekly_factor       float64
pricing_monthly_factor      float64
dtype: object

### Methodes :
##### separe des types
##### methode delete duplicated
##### enléve les NA
##### drop useless futures columns
##### deux types de quantitifs : catégories et les valeursa 
##### on pourra avoir les prix en focntion des arrondissements
##### methode appelle toutes

In [13]:
class FeatureRecipe:

    def __init__(self,data:pd.DataFrame):
        """ TODO : __init__ """

        self.data = data
        self.continuous = None # float
        self.categorical = None # Object
        self.discrete = None # int
        self.other = None
        self.datetime = None

    def separate_variable_types(self) -> None:
        """" TODO : Diviser les types de variables dans un tableau """
        print("-- Diviser les types de variables dans un tableau --")
        self.continuous = [] # float
        self.categorical = [] # Object
        self.discrete = [] # int   
        self.other = [] # other ( bool )
        for col in self.data.columns :
            if self.data[col].dtypes == int :
                self.discrete.append(self.data[col])
            elif self.data[col].dtypes == object :
                self.categorical.append(self.data[col])
            elif self.data[col].dtypes == float :
                self.continuous.append(self.data[col])
            else:
                self.other.append(self.data[col])
                          
        print(" The Amount of sepereted types:\n Categories {} \n Discrete {} \n Continuous {} \n Other {}".format(len(self.categorical),len(self.discrete),len(self.continuous),len(self.other)))
    
    def drop_uselessf(self):
        """ TODO : Supprimer les colonnes inutiles du dataset """
        print("-- Dropping uselese Columns--")

        if "Unnamed : 0" in self.data.columns :
            self.data.drop(['Unnamed: 0'],axis='columns',inplace=True)

        for col in self.data.columns :
            if self.data[col].isna().sum() == len(self.data[col]) :
                self.data.drop([col],axis='columns',inplace=True)

        print("--Done Dropping--")
    
    def deal_duplicate(self):
        """ TODO : Supprimer les lignes dupliquées du dataset """
        
        print("-- Duplicas Deletion commenced --")
        # Create an empty set 
        duplicateColumnNames = set() 
        # Iterate through all the columns of dataframe 
        for x in range(self.data.shape[1]): 
            # Take column at xth index. 
            col = self.data.iloc[:, x] 
            # Iterate through all the columns in DataFrame from (x + 1)th index to last index 
            for y in range(x + 1, self.data.shape[1]): 
                # Take column at yth index. 
                otherCol = self.data.iloc[:, y] 
                # Check if two columns at x & y index are equal or not, if equal then adding  to the set 
                if col.equals(otherCol): 
                    duplicateColumnNames.add(self.data.columns.values[y])
        
        #return list(duplicateColumnNames)
        self.data = self.data.drop(columns = list(duplicateColumnNames),inplace=True)

        print("-- Duplicas Deteted --")

    def drop_nanp(self, threshold: float):
        """ 
            TODO : Supprimer un pourcentage de NA du dataset 

            Threshold is inserted from 0 to 100 float variable
        """
        
        dropped=[]
        print("-- threshold est mis en Argument {}-- ".format(threshold))
        for col in self.data.columns :
            if (self.data[col].isna().sum()/self.data.shape[0]) >= threshold/100 : 
                dropped.append(col)
        print("{} feature have more than {} NaN ".format(len(col), threshold))
        print('\n\n - - - features - - -  \n {}'.format(col))    
        
        self.data.drop(dropped,axis='columns',inplace=True)
        print("-- Columns Dropped {}".format(dropped))
    
    def deal_dtime(self):
        """ TODO : Traiter les DateTime """
        pass

    def prepare_data(self, threshold: float):
        self.separate_variable_types()
        self.drop_uselessf()
        self.deal_duplicate()
        self.drop_nanp(threshold)
        self.deal_dtime()   
        print("-- End of FeatureRecipe --") 

In [14]:
fr = FeatureRecipe(d.merge)

In [15]:
fr.separate_variable_types()

-- Diviser les types de variables dans un tableau --
 The Amount of sepereted types:
 Categories 4 
 Discrete 5 
 Continuous 6 
 Other 5


In [56]:
fr.drop_uselessf()

-- Dropping uselese Columns--
--Done Dropping--


In [38]:
class FeatureExtractor:
    """
    Feature Extractor class
    """    
    def __init__(self, data: pd.DataFrame, flist: list):
        """
            Input : pandas.DataFrame, feature list to drop
            Output : X_train, X_test, y_train, y_test according to sklearn.model_selection.train_test_split
        """
        self.df = data
        self.flist = flist
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None

    def extract(self):
        print("-- X and y Values to be put while discarding the flist that we dont need--")
        for col in self.flist:
            if col in self.df:
                self.df.drop(col,axis="columns",inplace=True)

        print("-- Extraction Done --")

    def split(self,size:float,rand:int,y:str):
        print("-- Using Split mehtode --")
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.df.loc[:,self.df.columns!=y], self.df[y], test_size=size, random_state=rand)
        print("-- Afficher la shape de vos données --")

    def get_process_data(self,size:float,rand:int,y:str):
        self.extract()
        self.split(size,rand,y)
        print("-- Done processing Feature Extractor --")
        return self.X_train, self.X_test, self.y_train, self.y_test



  print("self.X_train.shape = {}, self.y_train.shape = {},\n\nself.X_test.shape = {}, self.y_test.shape = {}".format(self.X_train.shape, self.y_train.shape,self.X_test.shape, self.y_test.shape))
        
        print("-- créer l'objet de régression et entrainer le sur notre ensemble d'entraînement --")
        regr= LinearRegression()
        model= regr.fit(self.X_train,self.y_train)

        print("-- On affiche le vecteur des coefficients pour interpréter rapidement le modèle --")
        
        print("Model Coefficients so basicaly the X columns values for y values = {} , \n Model Intercept Value = {} , \n Score Result = {}".format(model.coef_ , model.intercept_, regr.score(self.X_test,self.y_test)))

        print("-- Test Train Split Done --")


In [49]:
flist = ['listing_id','city','neighborhood','latitude','longitude','is_rebookable','is_new_listing','is_fully_refundable','is_host_highly_rated','is_business_travel_ready','pricing_weekly_factor','pricing_monthly_factor','name','type']      


In [50]:
fe = FeatureExtractor(d.merge,flist)
X_train,X_test,y_train,y_test = fe.get_process_data(0.3,42,'local_price')

-- X and y Values to be put while discarding the flist that we dont need--
-- Extraction Done --
-- Using Split mehtode --
-- Afficher la shape de vos données --
-- Done processing Feature Extractor --


In [60]:
X_train

,Unnamed: 0,person_capacity,beds,bedrooms,bathrooms
728,746,4,2,1,1.0
630,649,2,1,1,1.0
394,415,6,4,3,1.5
777,791,2,2,0,1.0
598,617,2,1,1,1.0
...,...,...,...,...,...
106,131,1,1,1,1.0
270,291,6,3,2,1.0
860,877,4,2,2,1.0
435,456,5,1,1,1.0


In [59]:
y_train

728    298.366755
630    109.421875
394    330.000000
777     40.067010
598     70.434109
          ...    
106     40.234987
270    382.621359
860    115.000000
435    104.789474
102     92.281330
Name: local_price, Length: 699, dtype: float64

In [ ]:
class ModelBuilder:
    """
        Class for train and print results of ml model 
    """
    def __init__(self, model_path: str = None, save: bool = None):
        self.model_path=model_path
        self.save=save
        self.reg = LinearRegression()
    def __repr__(self): # class courier python , affichage par default isntancié , methode __str__ 
        pass
    def train(self,X,y):
        self.fit(X,y)
    def predict_test(self, X) -> np.ndarray:  # certain ligne of X_test or y_test 
        self.line_reg.fit(X,y)
    def predict_from_dump(self, X) -> np.ndarray:
        pass
    def save_model(self, path:str): # joblib saving de predict_test of fit , et on faire le dump de joblib par la focntion de predict_from_dump
        #with the format : 'model_{}_{}'.format(date)
        result = pickle.dump(reg)
        dump(result,"model_{}.joblib".format(DT.datetime.now()))
        pass
    def print_accuracy(self): # accuracy c'est le score 
        self.reg.predict(X_test)
        self.reg.score(X_test,y_test)
        pass
    def load_model(self): # à partir de fichier joblib , je le mets en instance 
        #self.model en plsu de model baf... si j'ai pas chargé alors pas de modéle  , s'il y a alors charge
        try:
            #load model
            pass
        except:
            pass

In [ ]:
def DataManager(d:DataHandler=None, fr: FeatureRecipe=None, fe:FeatureExtractor=None): # script model.py qui sort une model .joblib , alors dans model.py datamanager model builder et va sortir un fichier .joblib
    """
        Fonction qui lie les 3 premières classes de la pipeline et qui return FeatureExtractor.split(0.1)
    """
    pass
#on appelera la fonction DataManager() de la facon suivante : 
X_train, X_test, y_train, y_test = DataManager()

In [ ]:
m = ModelBuilder() 
m.train(X_train, y_train)
m.print_accuracy()
m.save_model(path)